In [ ]:
import numpy as np
%matplotlib notebook
from matplotlib import pyplot as plt
from BuildGeometry_v2 import BuildGeometry_v2
from brainweb import PETbrainWebPhantom

binPath = '/home/cc16/gits/apirl/build/bin'
temPath = '/tmp/lol'
geoPath = '/tmp/lol'
phanPath = '/tmp/lol'

PET = BuildGeometry_v2('mmr')

In [ ]:
img_3d_batch, mumap_3d_batch, t1_3d_batch, _ = PETbrainWebPhantom(
    phanPath, phantom_number=[0,1], voxel_size=np.array(PET.image.voxelSizeCm)*10, image_size=PET.image.matrixSize,
    pet_lesion=False, t1_lesion=False)

In [ ]:
# 2D PET --------------------------------------------------
PET.loadSystemMatrix(geoPath,is3d=False)

In [ ]:
img_2d = img_3d_batch[0,:,:,50]
mumap_2d = mumap_3d_batch[0,:,:,50]
img_2d_batch = img_3d_batch[:,:,:,50]
mumap_2d_batch = mumap_3d_batch[:,:,:,50]
psf_cm = 0.4

In [ ]:
# 3D PET ---------------------------------------------------
#PET.plotLorsAxialCoor()
#PET.plotMichelogram()
PET.setApirlMmrEngine(binPath=binPath, temPath=temPath, gpu=True)

In [ ]:
# 3D forward project
#y3d = PET.forwardProjectBatch3D(img_3d_batch[0,:,:,:],psf=psf_cm)
#y3d_batch = PET.forwardProjectBatch3D(img_3d_batch,psf=psf_cm)

# simulate 3D noisy sinograms
y3d,AF3d,_ = PET.simulateSinogramData(
    img_3d_batch[0,:,:,:], mumap=mumap_3d_batch[0,:,:,:], counts=100e6, psf=psf_cm)
##y3d_batch,AF3d_batch,_ = PET.simulateSinogramData(img_3d_batch, mumap = mumap_3d_batch, counts= 100e6, psf = psf_cm)

In [ ]:
# 3D OSEM and RAMLA reconstruction
img_osem_3d = PET.OSEM3D(y3d, AN=AF3d, niter=3, nsubs=14, psf=0.2)
##img_osem_3d_batch = PET.OSEM3D(y3d_batch, AN=AF3d_batch, niter = 2, nsubs = 6, psf=0.2)
#img_ramla_3d = PET.Ramla3D(y3d, AN=AF3d, niter = 2, nsubs = 4, psf=0.2)
#img_ramla_3d_batch = PET.Ramla3D(y3d_batch, AN=AF3d_batch, niter = 2, nsubs = 6, psf=0.2)

# switch back to 2D
##PET.is3d = False

In [ ]:
from caspyr.plotting import volshow
#vmax=img_osem_3d.mean() * 100
volshow([img_osem_3d[90:-90, 90:-90, ::-1].transpose(1,2,0)]);